In [2]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.insert(0, os.path.abspath('../'))
import os
import time

import numpy as np

np.set_printoptions(formatter={'int':hex})

from PyPci import pci
dev = pci.pci()
#####################################################################

LEN       = 128*32     #128*n = n KB
ADDR      = 0x000

In [ ]:
dev.rstBAR(0)
#dev.rstBAR(1)

In [ ]:
#data  = np.random.randint(0,0xfffffffffffffff,LEN)
#data = np.arange(LEN, dtype=np.uint8)
#data = np.frombuffer(data.tobytes(), dtype=np.uint64)
#data1 = (data.astype(np.uint64)) << 4
#dataDIO = ((data1 & 0x000000000000) | 0x0000000000000000)
#dataDMA = ((data1 & 0x000000000000) | 0x6666777788889999)

def task_writeDIO(addr, data):
    dev.write(data, addr)
    
def task_readDIO(addr, length):
    data2 = dev.read(addr, length)
    for i in range (length):
        print hex(i), "\t:", hex(data2[i])
        
def task_writeDMA():
    dev.writeDMA(data1, 0x00)
    
def task_readDMA(length, add_ptr):
    data3 = dev.readDMA(length, add_ptr)
    print data3
    #for i in range (0,length):
    #    print hex(i), "\t:", hex(data3[i]), hex(data1[i])
    #if (data1 == data3).all():
    #    print 'Matched'
task_readDMA(100, 0)
task_readDIO(0, 1)

In [6]:
%%writefile cluster.py
from PyPci import pci
import numpy as np
from PyPci import pci
import time
np.set_printoptions(formatter={'int':hex})

class fifo:
    def __init__(self):
        self.dev = pci.pci()
        self.sendptr = 1
        self.recieveptr = 1
        pci_command = np.array([0xFFFFFFFF000000FE], dtype=np.uint64)
        self.dev.write(pci_command, 0x00)
        pci_status = self.dev.read(0x00, 1)
        print 'pci_status :', hex(pci_status)
        print 'pci_command :', hex(pci_command)
        pci_command = np.array([(self.recieveptr << 20) ^ (self.sendptr << 8)], dtype=np.uint64)
        self.dev.write(pci_command, 0x00)
        pci_status = self.dev.read(0x00, 1)
        print 'pci_status :', hex(pci_status)
        print 'pci_command :', hex(pci_command)
        return
    
    def reset():
        self.sendptr = 1
        self.recieveptr = 1
        pci_command = np.array([0xFFFFFFFF000000FE], dtype=np.uint64)
        self.dev.write(pci_command, 0x00)
        pci_status = self.dev.read(0x00, 1)
        print 'pci_status :', hex(pci_status)
        print 'pci_command :', hex(pci_command)
        pci_command = np.array([(self.recieveptr << 20) ^ (self.sendptr << 8)], dtype=np.uint64)
        self.dev.write(pci_command, 0x00)
        pci_status = self.dev.read(0x00, 1)
        print 'pci_status :', hex(pci_status)
        print 'pci_command :', hex(pci_command)
        return 0
    
    def send_packet(self, packet):
        #print '>>>'
        #print packet
        pci_status = self.dev.read(0x00, 1)
        pciptr = (int(pci_status[0]) & 0x00000000FFF00000) >> 20
        fifoptr = (int(pci_status[0]) & 0x00000000000FFF00) >> 8
        #print 'recieveptr:', hex(self.recieveptr), 'sendptr:', hex(self.sendptr)
        #print 'pciptr:', hex(pciptr), 'fifoptr:', hex(fifoptr)
        ##
        ## Some issue with the fifo implemented on the pci side in cluster
        ##
        endptr = self.sendptr + packet.size
        #thresh = 0xFFF - self.sendptr
        #print '$', hex(endptr)
        if endptr > 0xFFF:
            partpacket0 = packet[:(0x1000 -self.sendptr)]
            partpacket1 = packet[(0x1000 -self.sendptr):]
            self.dev.writeDMA(partpacket0, self.sendptr)
            self.dev.writeDMA(partpacket1, 1)
            self.sendptr = (self.sendptr + packet.size + 1)& 0x1FFF
        else:            
            self.dev.writeDMA(packet, self.sendptr)
            self.sendptr = (self.sendptr + packet.size)& 0x1FFF
            
        if pciptr < self.sendptr:
            self.sendptr = self.sendptr & 0xFFF
        pci_command = np.array([((self.recieveptr& 0xFFF) << 20) ^ ((self.sendptr& 0xFFF) << 8)], dtype=np.uint64)
        self.dev.write(pci_command, 0x00)
        #print 'recieveptr:', hex(self.recieveptr), 'sendptr:', hex(self.sendptr)
        #print 'pciptr:', hex(pciptr), 'fifoptr:', hex(fifoptr)
        return 0
    
    def recieve_packet(self):        
        data = np.array([], dtype=np.uint64)
        pci_status = self.dev.read(0x00, 1)
        pciptr = (int(pci_status[0]) & 0x00000000FFF00000) >> 20
        fifoptr = (int(pci_status[0]) & 0x00000000000FFF00) >> 8
        data = np.array([], dtype=np.uint64)
        #print 'recieveptr:', hex(self.recieveptr), 'sendptr:', hex(self.sendptr)
        #print 'pciptr:', hex(pciptr), 'fifoptr:', hex(fifoptr)
        #print hex(pci_status)
        while 1:
            if fifoptr < self.recieveptr:
                buffsize = 0x1000 - self.recieveptr
            else:
                buffsize = fifoptr - self.recieveptr
            #print '<<', hex(buffsize), hex(self.recieveptr) 
            if buffsize:
                data = np.append(data, self.dev.readDMA(buffsize, self.recieveptr))
                self.recieveptr = (self.recieveptr + buffsize) & 0xFFF
                if self.recieveptr == 0:
                    self.recieveptr = 1
            else:
                break
            
        pci_command = np.array([((self.recieveptr& 0xFFF) << 20) ^ ((self.sendptr& 0xFFF) << 8)], dtype=np.uint64)
        self.dev.write(pci_command, 0x00)
        #print 'recieveptr:', hex(self.recieveptr), 'sendptr:', hex(self.sendptr)
        #print 'pciptr:', hex(pciptr), 'fifoptr:', hex(fifoptr)
        return data
    
    
class cluster:
    def __init__(self):
        self.Fifo = fifo()
        self.queue = []
        self.olddata = np.array([], dtype=np.uint64)
        self.nodes = 0
        return
    
    def reset(self):
        self.Fifo.reset
        self.queue = []
        self.olddata = np.array([], dtype=np.uint64)
        self.nodes = 0
        return 0
        
    def conf(self, nodes):
        self.nodes = nodes
        for i in range(self.nodes):
            #self.reset_node(0xFFFF)
            #time.sleep(1)
            print 'setting node %d with address : %d' %(i, i*4)
            self.set_node_address(0xFFFF, i*4 & 0xFFFF)
            #time.sleep(1)
            
    
    def encapsulate_packet(self, source, sink, packet, timeout):
        return np.insert(packet, 0, 0xFF ^ sink << 8 ^ source << 24 ^ len(packet) << 40 ^ timeout << 56 ^ 0xF << 60)
    
    def reset_node(self, node_index):
        rst_packet = np.array([0xf400010000FFFFFF, 0xFFFFFFFFFFFFFFFF, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.uint64)
        rst_packet[0] = (int(rst_packet[0]) & 0xFFFFFFFFFF0000FF) ^ (node_index << 8)
        #print rst_packet
        self.Fifo.send_packet(rst_packet)
        self.queue = []
        return 0
    
    def set_node_address(self, node_index, new_node_index):
        set_packet = np.array([0xf400010000FFFFFF, 0x000000000000AADD, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.uint64)
        set_packet[0] = (int(set_packet[0]) & 0xFFFFFFFFFF0000FF) ^ (node_index << 8)
        set_packet[1] = (int(set_packet[1]) & 0xFFFFFFFF0000FFFF) ^ (new_node_index << 16)
        #print set_packet
        self.Fifo.send_packet(set_packet)
        return 0
    
    def sendpackets(self, packet, source=0xFFFC, sink=0xFFFF):
        zeros = np.zeros((10,), dtype=np.uint64)
        buff = self.encapsulate_packet(source, sink, packet, 0x08)
        #print buff
        self.Fifo.send_packet(np.append(buff, zeros))
        #print buff
        return 0
    
    def getheader(self, packet):
        for i in range(packet.size):
            if (int(packet[i]) & 0xFF) == 0xFF:
                source   = (int(packet[i])   & 0xFFFF000000) >> 24
                sink     = (int(packet[i])   & 0xFFFF00) >> 8
                size     = (int(packet[i])   & 0xFFFF0000000000) >> 40
                packetid = (int(packet[i+1]) & 0xFFFF000000000000) >> 48
                #print 'packetid:', packetid
                #print 'size:', size
                #print 'packet.size:', packet.size
                if size < packet.size - 1:
                    return i+1, size, sink, source, packetid
                else:
                    return packet.size, 0, 0, 0, 0
        return packet.size, 0, 0, 0, 0
    
    def getpackets(self, idlist):
        data = np.append(self.olddata, self.Fifo.recieve_packet())
        #print '#######'
        #print data
        #print '#######'
        packets = []
        #print data.size
        pktptr = 0
        while pktptr < data.size:
            #print pktptr, data.size
            i, size, sink, source, packetid = self.getheader(data[pktptr:])
            #print i, size, hex(sink), hex(source), hex(packetid)
            if size:
                pkt = np.copy(data[pktptr+i:pktptr+i+size])
                element = {'size':size, 'sink': sink, 'source':source, 'packetid':packetid, 
                           'packet':pkt}
                self.queue.append(element)
                pktptr = pktptr+i+size
            else:
                break
        self.olddata = data[pktptr:]
        #print self.queue    
        for i in idlist:
            try:
                e = (item for item in self.queue if item['packetid'] == i).next()
                element = {'size':e['size'], 'sink': e['sink'], 'source':e['source'], 'packetid':e['packetid'], 
                           'packet':e['packet']}
                packets.append(element)
                #print i, element
                self.queue.remove(e)
            except StopIteration:
                #print "packetid not found !!"
                pass
            #print packets
        return packets
        

Overwriting cluster.py


In [5]:
respacket = Cluster.getpackets(pid)
print packet
print respacket
for i in respacket:
    print i['packetid']
print pid

NameError: name 'Cluster' is not defined

In [7]:
%reload_ext autoreload
%autoreload 2
import numpy as np
np.set_printoptions(formatter={'int':hex})


import cluster
import aescore
import time

Cluster = cluster.cluster()
Cluster.conf(4)

PCI is being Initialized....
PCI Initialized
BAR0 Size: 0x10000
BAR1 Size: 0x100000
pci_status : 0xffffffffa1a011feL
pci_command : 0xffffffff000000feL
pci_status : 0xa1a01100L
pci_command : 0x100100L
setting node 0 with address : 0
setting node 1 with address : 4
setting node 2 with address : 8
setting node 3 with address : 12


In [10]:
for i in range(10):    
    packetid, databuff, cryptbuff = aescore.burst_pairs_test(size = 8)
    #pid.append(packetid)
    #packetid, databuff, cryptbuff = aescore.burst_pairs_test(size = 8)
    #print 'packetid:', hex(packetid) #, databuff, cryptbuff
    #print databuff
    Cluster.sendpackets(databuff, source=0x0000, sink=0x0004)
    #time.sleep(0.5)
    retbuff = Cluster.getpackets([packetid])
    if len(retbuff):
        if (retbuff[0]['packet'][1:] == cryptbuff).all():
            print "encryption is correct"
            pass
        else:
            print retbuff[0]['packet'][1:]
            print cryptbuff
            print '@',i,'@'
            pass
    else:
        print i,

encryption is correct
[0x59c96e9d1e883e30L 0xda6f18162e29da75L 0x2006f65a1a636dfeL
 0xb622f11fef54ca2L 0x553699436d999eb9L 0x258b6ef3ff206942L
 0xea450880ef0073f8L 0xe7af89293edb6554L 0xb1ab2e8d34e44e21L
 0x13656b6013ca358aL 0x134322a86bad83c6L 0x18bd81f0d4a35efdL
 0x29ab1bbec8eaabe8L 0xc752924b4803bc5bL 0x60c4e356a6fedb3aL 0x0L]
[0x59c96e9d1e883e30L 0xda6f18162e29da75L 0x2006f65a1a636dfeL
 0xb622f11fef54ca2L 0xc9b483a8d9cd1c4bL 0x553699436d999eb9L
 0x258b6ef3ff206942L 0xea450880ef0073f8L 0xe7af89293edb6554L
 0xb1ab2e8d34e44e21L 0x13656b6013ca358aL 0x134322a86bad83c6L
 0x18bd81f0d4a35efdL 0x29ab1bbec8eaabe8L 0xc752924b4803bc5bL
 0x60c4e356a6fedb3aL]
@ 1 @
encryption is correct
encryption is correct
encryption is correct
encryption is correct
encryption is correct
encryption is correct
encryption is correct
encryption is correct


In [11]:
packetid, databuff, cryptbuff = aescore.burst_pairs_test(size = 8)
print 'packetid:', hex(packetid) #, databuff, cryptbuff
Cluster.sendpackets(databuff, source=0x0000, sink=0x000c)
retbuff = Cluster.getpackets([packetid])
if len(retbuff):
    if (retbuff[0]['packet'][1:] == cryptbuff).all():
        print "encryption is correct"

packetid: 0x81e0


In [13]:
retbuff = Cluster.getpackets([packetid])
if len(retbuff):
    if (retbuff[0]['packet'][1:] == cryptbuff).all():
        print "encryption is correct"

In [14]:
import numpy as np
pid = []
buff = np.array([], dtype=np.uint64)
zeros = np.zeros((5,), dtype=np.uint64)
for i in range (5):
    packetid, databuff, cryptbuff = aescore.burst_pairs_test(size = 8)
    pid.append(packetid)
    #print 'packetid:', hex(packetid) #, databuff, cryptbuff
    Cluster.sendpackets(databuff, source=0x0000, sink=0x0005)
    packetid, databuff, cryptbuff = aescore.burst_pairs_test(size = 8)
    pid.append(packetid)
    #print 'packetid:', hex(packetid) #, databuff, cryptbuff
    Cluster.sendpackets(databuff, source=0x0000, sink=0x0004)
    
retbuff = Cluster.getpackets(pid)
print len(retbuff)


0


In [17]:
%reload_ext autoreload
%autoreload 2
import numpy as np
np.set_printoptions(formatter={'int':hex})


import cluster
import aescore
import time

Cluster = cluster.cluster()
Cluster.conf(3)

PCI is being Initialized....
PCI Initialized
BAR0 Size: 0x10000
BAR1 Size: 0x100000
pci_status : 0xffffffff001001feL
pci_command : 0xffffffff000000feL
pci_status : 0x100100L
pci_command : 0x100100L
setting node 0 with address : 0
setting node 1 with address : 4
setting node 2 with address : 8


In [27]:
import numpy as np

pkt = [0xc00a0000540381aeL, 0x9def929dac828df3L, 0x3204c000000489f4L,
       0xc36e7508181b6369L, 0xc35b429b84afa310L, 0x6b65687369686241L,
       0x2e6961706a614220L, 0xa728264386dfeb5fL, 0xde89881d4b8dc06fL]
#0x16e00000540381aeL 0x75fa28c42e017c5aL 0x790c000000eb4f7L
#0x54fc2676f6fe1187L 0x4b8b9dc6c5c5f7b0L 0x6b65687369686241L
#0x2e6961706a614220L 0xfe6e62d7dfbfa5cbL 0x2f49ad06504a0a71L

key = np.array([0x9def929dac828df3L, 0x3204c000000489f4L, #0x3204e7f0000489f4L,
                0xc36e7508181b6369L, 0xc35b429b84afa310L], dtype=np.uint64)
data = np.array([0x6b65687369686241L, 0x2e6961706a614220L], dtype=np.uint64)
crypto = np.array([0xa728264386dfeb5fL, 0xde89881d4b8dc06fL], dtype=np.uint64)

packetid, databuff = aescore.search_key(key, data, crypto, searchspace = 0x54)

print databuff
Cluster.sendpackets(databuff, source=0x0000, sink=0x000a)
for i in range(10):
    time.sleep(1)
    retbuff = Cluster.getpackets([packetid])
    if len(retbuff):
        print retbuff
        break

[0x2f860000540381aeL 0x9def929dac828df3L 0x3204c000000489f4L
 0xc36e7508181b6369L 0xc35b429b84afa310L 0x6b65687369686241L
 0x2e6961706a614220L 0xa728264386dfeb5fL 0xde89881d4b8dc06fL]
[{'source': 10L, 'packetid': 12166L, 'sink': 0L, 'packet': array([0x2f8600ffff0381eaL, 0x27fcbdaL, 0x27fcbdaL, 0x27fcbdaL, 0x27fcbdaL,
       0x27fcbdaL, 0x27fcbdaL, 0x27fcbdaL, 0x27fcbdaL, 0x27fcbdaL], dtype=uint64), 'size': 10L}]


In [1]:
import numpy as np

pkt = [0xc28f0000800381aeL, 0x75b5f1d72b7f27c5L, 0x874992710f6c423cL,
       0xe12f153ddc000000L, 0x1c395558fa7f8c64L, 0x6b65687369686241L,
       0x2e6961706a614220L, 0xff46ac202fb481b0L, 0xa5508d07bee2e397L]

packetid, databuff = (int(pkt[0])>> 48)& 0xFFFF, np.array(pkt, dtype=np.uint64)

print databuff
Cluster.sendpackets(databuff, source=0x0000, sink=0x0001)
for i in range(10):
    time.sleep(1)
    retbuff = Cluster.getpackets([packetid])
    if len(retbuff):
        print retbuff
        break

[14019424167175356846  8481951379672213445 9748483882707862076
 16226211337867362304  2033750547382635620  7738706379605041729
  3344311333748294176 18394578982280659376 11912176078801789847]


NameError: name 'Cluster' is not defined

In [9]:
import aescore
key = np.array([0xa8c85134820ebd32, 0xfff2c612c1b97649L,
          0x50fb58d9b7587eb4L, 0x84827b92d571ab8cL], dtype=np.uint64)
data = np.array([0x6b65687369686241L, 0x2e6961706a614220L], dtype=np.uint64)
packetid, databuff = aescore.burst_pairs(key, data, size = 8)

Cluster.sendpackets(databuff, source=0x0000, sink=0x0001)
for i in range(10):
    time.sleep(1)
    retbuff = Cluster.getpackets([packetid])
    if len(retbuff):
        print retbuff
        break


[0x25369452030125c6L 0xe3f805230bbff529L]
[0x25369452030125c6L 0xe3f805230bbff529L]
[0x25369452030125c6L 0xe3f805230bbff529L]
[0x25369452030125c6L 0xe3f805230bbff529L]
[0x25369452030125c6L 0xe3f805230bbff529L]
[0x25369452030125c6L 0xe3f805230bbff529L]
[0x25369452030125c6L 0xe3f805230bbff529L]
[0x25369452030125c6L 0xe3f805230bbff529L]
[{'source': 1, 'packetid': 37404L, 'sink': 0, 'packet': array([0x921c0000080181aeL, 0xa8c85134820ebd32L, 0xfff2c612c1b97649L,
       0x50fb58d9b7587eb4L, 0x84827b92d571ab8cL, 0x6b65687369686241L,
       0x2e6961706a614220L, 0xa8c85134820ebd32L, 0xfff2c612c1b97649L,
       0x50fb58d9b7587eb4L, 0x84827b92d571ab8cL, 0x6b65687369686241L,
       0x2e6961706a614220L, 0xa8c85134820ebd32L, 0xfff2c612c1b97649L,
       0x50fb58d9b7587eb4L, 0x84827b92d571ab8cL, 0x6b65687369686241L,
       0x2e6961706a614220L, 0xa8c85134820ebd32L, 0xfff2c612c1b97649L,
       0x50fb58d9b7587eb4L, 0x84827b92d571ab8cL, 0x6b65687369686241L,
       0x2e6961706a614220L, 0xa8c85134820ebd32L,